<a href="https://colab.research.google.com/github/blue-create/langlens/blob/main/to_publish/extract_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Unzippen und Extrahieren von Daten
In diesem Notebook werden die Zip-Dateien extrahiert und gelesen und nach einem ersten Filter nach Land und Inhalt gespeichert.

Schritte:
1. Unzippen der XMLs in allen Ordnern
2. Konvertieren der XMLs zu CSV, mit erstem Filter:
  - nur deutsche Zeitungen, in deutscher Sprache
  - nur Artikel über häusliche Gewalt (nach Keyword-Search)

### Imports, Konstanten, Paths

In [ ]:
from google.colab import drive
import os
import zipfile
import pandas as pd
import xml.etree.ElementTree as ET
from tqdm import tqdm
import re

In [ ]:
# Definieren der Paths
RAW="0_Raw"
UNZIPPED="1_Unzipped"
FILTERED="2_Filtered/230818"

In [ ]:
# Verbinden mit GDrive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/data/

## Unzippen der Dateien
- XMLs in Ordnern werden extrahiert und gespeichert

In [ ]:
#Überprüfen der Anzahl der Dateien
num_files = len(os.listdir(RAW))
print("Anzahl der Zeitungs-Ordner: ", num_files)

Anzahl der Zeitungs-Ordner:  257


In [ ]:
## Extrahieren aller XMLs in jedem Zeitungs-Ordner
for filename in os.listdir(RAW)
    if filename.endswith(".zip"):
        if zipfile.is_zipfile(os.path.join(RAW, filename)):
            with zipfile.ZipFile(os.path.join(RAW, filename), "r") as zip_ref:
                zip_ref.extractall(UNZIPPED)
        else:
            print(f"Datei {filename} ist keine Zip-Datei und wird nicht extrahiert.")

In [ ]:
## Überprüfen ob alle Zip-Dateien extrahiert wurden
for filename in os.listdir(RAW):
    if filename.endswith(".zip"):
        name = filename.split(".")[0]
        for xml_filename in os.listdir(UNZIPPED):
            if xml_filename.startswith(name) and xml_filename.endswith(".xml"):
                break
        else:
            print(f"Es gibt keine XMLs-Datei für {filename}")

## Lesen der XMLs und erstes Filtern der Artikel und Titel

### Dictionarys zum Filtern
- diese Keyword-Liste wurde in einem iterativen Prozess erstellt um alle Artikel zum Thema Partnerschaftsgewalt zu erhalten

In [ ]:
# Definieren der Schlagworte zur ersten Keyword-Search der Artikel
comb=[
    "häuslicher? gewalt",
    "ehedrama",
    "familienstreit und partner(in)?",
    "gewalttätige[rn]? (ex-)?partner(in)?",
    "gewalttätige[rn]? Ehe(partner|mann|frau)?(in)?",
    "partnerschaftsgewalt",
    "femizid",
    "gewalt in (der )?(\(ex-\))?(ex)?partnerschaft(en)?",
    "beziehungsgewalt",
    "beziehungstat",
    "gewalttätige[rn]? (ex(-)?)?freund(in)?",
    "beziehungsdrama",
]

In [ ]:
# Definieren der Schlagworte dich nicht in relevanten Artikel auftauchen
exclude_titles=[
  #LESERBRIEFE
  "lesermeinung",
  "leserbrief",
  "LIEBE LESERIN, LIEBER LESER",
  #ANDERE KATEGORIEN
  "Corona-Ticker"
  "EM-Tabelle",
  "finanztrends",
  "News Ticker",
  "TV-Programm",
  "TV-"
  "Kriminalhörspiel",
  "TV-Tipp des Tages",
  "thriller"
]

### Methoden


In [ ]:
# Funktion zum Lesen von XML-Dateien und speichern aller Artikel in einer Liste
def parse_xml_file(xml_file):
  """ Funktion, die alle XMLs einer Zeitung liest und die Artikel mit Metadaten extrahiert
  Parameters:
    - prefix (str): Kürzel der Zeitung
  Returns:
    - json: Kombinierte Liste der Artikel
  """
  tree = ET.parse(os.path.join(UNZIPPED, xml_file))
  root = tree.getroot()
  json_file = []
  for artikel in root.findall('artikel'):
    artikel_id = artikel.find('metadaten/artikel-id')
    artikel_id = artikel_id.text if artikel_id is not None else None
    name = artikel.find('metadaten/quelle/name').text
    jahrgang = artikel.find('metadaten/quelle/jahrgang')
    jahrgang = jahrgang.text if jahrgang is not None else None
    autor = artikel.find('metadaten/autor/autor-name')
    autor = autor.text if autor is not None else None
    datum = artikel.find('metadaten/quelle/datum')
    datum = datum.text if datum is not None else None
    ressort_elem = artikel.find('inhalt/titel-liste/ressort')
    ressort = ressort_elem.text if ressort_elem is not None else None
    titel_elem = artikel.find('inhalt/titel-liste/titel')
    titel = titel_elem.text if titel_elem is not None else None

    untertitel_elem = artikel.find('inhalt/titel-liste/untertitel')
    untertitel = untertitel_elem.text if untertitel_elem is not None else None

    text = []
    text_elem = artikel.find('inhalt/text')
    try:
        p_elems = text_elem.findall('p')
        for p_elem in p_elems:
            p_text = p_elem.text
            if p_text is not None:
              text.append(p_text)

    except:
        pass
    temp_dict = {}
    temp_dict['artikel_id'] = str(artikel_id)
    temp_dict['name'] = name
    temp_dict['jahrgang'] = jahrgang
    temp_dict['datum'] = datum
    temp_dict['ressort'] = ressort
    temp_dict['titel'] = titel
    temp_dict['untertitel'] = untertitel
    temp_dict['text'] = text
    temp_dict['autor'] = autor

    json_file.append(temp_dict)
  return json_file


In [ ]:
#Funktion zum Filtern von Artikeln während des Extrahierens
def filter_text_by_list(text, list_match):
  """  Funktion um zu vergleichen ob minestens ein Wort in einer Liste von Wörtern in einem Text vorkommt und keines einer zweiten Liste von Wörtern
  Parameters:
    - text (str or list of str): Text der überprüft wird
    - list_match (list of str): Liste von Wörtern, nach denen gesucht wird
    - list_no_match (list of str): Liste von Wörtern, die nicht vorkommen dürfen
  Returns:
    - boolean:
      - True: wenn min. eines der ersten Liste, und keines der zweiten Liste von Wörtern im Text vorkommt
      - False: wenn keins der ersten oder min ein Wort der zweiten Wörterliste im Text vorkommen
  """
  text=" ".join(text).lower()

  for comb in list_match:
    if all(occurs(word,text)  for word in comb.split(" und ")):
      return True
  return False

In [ ]:
# Funktion zum überprüfen ob Datei schon exportiert wurde
def check_if_exported(prefix, filtered_path):
  """ Funktion zum Überprüfen, ob die Dateien für eine Zeitung schon exportiert wurden
  Parameters:
    - prefix (str): Kürzel der Zeitung
    - filtered_path: Path, in dem gesucht wird
  Returns:
    - boolean: True, wenn die Dateien für die Zeitung schon exportiert wurde, sonst False
  """
  filtered=os.listdir(filtered_path)
  filtered = [file.split(".")[0]for file in filtered if file.endswith(".csv")]
  if prefix in filtered:
    print(f"{prefix} csv already exported")
    return True
  else:
    return False

In [ ]:
# Funktion zum prüfen ob ein Schlagwort in einem Text vorkommt
def occurs(word, text):
  """ Funktion zum Überprüfen, ob Wort in Text auftaucht
  Parameters:
    - word (str): Wort nach dem gesucht wird
    - text (str): Text in dem gesucht wird
  Returns:
    - boolean: True, wenn das Wort in dem Text gefunden wird, sonst False
  """
  if len(re.findall(word,text))>0:
    return True
  else:
    return False


In [ ]:
# Funktion zum Filtern von Artikel basieren auf dem Titel
def filter_title(title):
  """ Funktion zum Filtern eines Artikels nach Titel (mit Liste: exclude_titles)
  Parameters:
    - title (str): Titel der überprüft werden soll
  Returns:
    - boolean: True, wenn
        - kein Titel
        - der Titel ist nicht in der Liste exclude_titles
  """
  if type(title)!=str:
    return True
  for ex in exclude_titles:
    if ex.lower() in title.lower():
      return False
  return True

### Filter nach Land
Alle nicht deutschen und nicht-deutschsprachigen Zeitungen werden ausgeschlossen.

In [ ]:
# Erstellen einer Liste der Kürzel der relevanten Zeitungen
all_prefixes= sorted([i.split(".")[0] for i in os.listdir(RAW)])
prefixes_to_exclude=['AGZ', "ANEW", "APZT","AWP", "AWPO", "AWPU", "BAZ", "BERN", "BEOB", "BLI", "BUND", "BWAI", "DIEW", "DOL", "ELNA","HZ", "HZO", "KLEI", "KRON", "KUR", "LUXT", "LZLZ", "NECH","NLZ", "NVB","NVT","NZZ", "NZZS", "OOEN", "PBN", "PRE", "PROF", "RVZ","SBLI","SN","STA","STG","TAG","TAS","THTA","TITA","VN","WEWO","WZ","ZSAS", "NBPC","NBPC_part1","NBPC_part2","NBPC_part3","NBPC_part4","FALT","RTAL"]
prefixes= sorted([i.split(".")[0] for i in os.listdir(RAW)])
prefixes=list(set(prefixes) - set(prefixes_to_exclude))

### Filtern nach Key-Words

In [ ]:
# Für alle relevanten Zeitungen:
# XML lesen, nach Titel und Artikel Text Filtern und resultierende Artikel als CSV speichern

for prefix in tqdm(prefixes):
  if not check_if_exported(prefix, FILTERED):
    xmls=[i for i in os.listdir(UNZIPPED) if i.startswith(prefix+"_")]
    DV_art=[]
    for xml in xmls:
      parsed_xml=parse_xml_file(xml)
      for art in parsed_xml:
        if filter_title(art["titel"] ):
          if filter_text_by_list(art["text"],comb):
              DV_art.append(art)
    DV_art=pd.DataFrame(DV_art)
    DV_art=DV_art.drop_duplicates("text")
    DV_art.to_csv(FILTERED+"/"+prefix+".csv")